## Packages
Before we start, we must import all the needed packages. Below are the packages we will use.
### Packages for Webscraping

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re
import pandas as pd 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Packages to build and interact with the interface

In [2]:
import tkinter as tk
from tkinter import ttk
import customtkinter
import webbrowser
import ipywidgets as widgets

### Packages for sending email

In [3]:
import smtplib
from email.message import EmailMessage
from tkinter import messagebox # success saving criteria/sending mail or not
from IPython.display import display, HTML

## Part 1 : webscraping
There are many ways to scrape data from a job posting website. Depending on the website structure and where the required data can be found, we may  need to scrape only the main page,  or we may need to click on each job posting and be directed to a new url to find the 
data. Furthemore, we have to take into consideration that job postings typically span multiple pages, and there exists many ways to iterate over the 
pages (by url, by clicking on the 'next page' button', loading more jobs on the same page button, etc). 

### A) Collecting links for each job posting 
Before we start, we need to identify what type of data we want to scrape. Since our ultimate goal is to allow the user to apply directly on the company's website (either after receiving the jobs by mail, or by searching on our interface), a crucial element is the company's website url, which is typically found after we click on the individual job posting of the main page and get redirected to the detailed job posting website. 

**Problem** : if we decide to scrape the data by clicking on each job posting, extracting information, then coming back to the main page, we receive the'Stale element' error notification, which is probably caused by the dynamically changing nature of the website (elements sometimes go missing after we quit then come back to the same page)

**Solution** : instead of clicking, we can extract the href link that takes us to the detailed job posting, which can be scraped from the main page on Indeed. We repeat this process for each job posting on the page, and store them all in a list (job_details_links)

Once we do this for the first page, we have to iterate over other pages. We find that clicking on the 'next page button isn't always optimal because
most of the times we will receive an error saying the element was intercepted, and another element was clicked instead. On another hand, Indeed
doesn't have a 'load more jobs' button, so we remove this possibility. 

In [4]:
job_details_links = []

url = "https://fr.indeed.com/jobs?q=data&l=&vjk=23d23734ba242a27"
driver_path = print(ChromeDriverManager().install())  
driver = webdriver.Chrome(driver_path)
driver.get(url)

while True:
    job_postings_elements = driver.find_elements(By.CLASS_NAME, "css-zu9cdh")
    for job in job_postings_elements:
        for i in range(1, 18): #the xpath will go around 17 'li' in each page
            if i % 6 == 0: # Specifically, there are 15 jobs per pages, but in instances li(6) and li(12), the xpath represents an ad instead of a job posting, hence the range (1,18) and the % 6
                continue
            xpath = "/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{}]/div/div/div/div/div/table[1]/tbody/tr/td[1]/div[1]/h2/a".format(i)
            element = job.find_elements(By.XPATH, xpath)
            if element:
                job_details_link = element[0].get_attribute('href')
                job_details_links.append(job_details_link)
                
    # However, each next page button is identified with an href that takes us to the next page. Hence, we scrape this href. However, it is important 
    # to note that we identify the element of the next page button not with class name (because class is the same for the next and previous page button,
    # and we enter in an infinite loop :)), but with its unique CSS selector (data-testid='pagination-page-next'). 
    # We could also do this with its Xpath, however, part of the Xpath, associated with li will change with each page (for ex page 1 will be li[1], 
    # page 2 ;i[2] and so on, so we would have to sepcify for i in range[1:page_numbers], and specify the Xpath with li[{}] and wiht '.format(i)' at the end. 
    
    next_button = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='pagination-page-next']")
    if not next_button: 
        print("No more pages available.")
        break
    
    next_button_link = next_button[0].get_attribute("href")
    driver.get(next_button_link)

print("Number of job details links found:", len(job_details_links))

C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
No more pages available.
Number of job details links found: 955


<font color="red">
    
**Note:** it is possible for the website to ask to verify you are human, which will require manual intervention and render the code obsolete. To fix this, switch networks, preferraby to a private one such as personal hotspot.

<font>

### B) Scraping each link
Once we have gathered all the job postings links and stored them in the 'job_details_links' list, we create a function to scrape specific data based on their elements (class_name, XPATH, css selector). Indeed, the criterias are identified with the same elements in all job_links which is convenient.
These criteria include:
- **Job title**
- **Company name***
- **City**
- **Postal code**
- **Salary**
- **Type of remote work allowed**
- **Apply button URL**

Scraped data is stored in the dictionary 'job_details', where the data is the value and the name of the criteria (title, company, etc) is the key.
We use the try function here instead of regular if because it is useful for debugging more efficiently in case of an error


In [6]:
def scrape_job_details(url):
    driver_path = print(ChromeDriverManager().install())  
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    
    #Initializing an empty dictionary to store job details
    job_details = {}           
    
    try:
        # Extracting job title
        job_details['title'] = driver.find_element(By.CLASS_NAME, 'jobsearch-JobInfoHeader-title').text
        
        # Extracting company name
        job_details['company'] = driver.find_element(By.CLASS_NAME, 'css-hon9z8').text

        # Next we would like to extract the location and the postal code of the job posting. However, a few specifications should be made in this regard.
        
        # On Indeed, city and postal code share the same element (whether it be CLASS_NAME, XPATH or other), so we have to separate them. We will do so
        # by identifying if the characters in the 'location' element contain digits, if so, we wil take the part of 'location' associated with digits 
        # and assign it to postal code, and the rest will be associated to city (meaning the part that is not digits).
        
        location_element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="inlineHeader-companyLocation"]').text
        if any(char.isdigit() for char in location_element):
            job_details['postal_code'] = ''.join(char for char in location_element if char.isdigit())
            
            # Some postal codes are expressed with 5 numbers, while others are expressed with 2. To garantee a clean and consistent code, we restrict 
            #all postal codes to their first 2 numbers. 
            
            job_details['postal_code'] = job_details['postal_code'][:2]
            job_details['city'] = ''.join(char for char in location_element if not char.isdigit())
           
            # Replacing parentheses remaining from extracting postal codes with spaces in city if they exist.
            
            if '()' in job_details['city']:
                job_details['city'] = job_details['city'].replace('()', ' ')
        
        # If no digits are found, then we assume there is no postal code specified and we assign the location element directly to city.
        
        else:
            job_details['postal_code'] = "NA"
            job_details['city'] = location_element
                
        

        # Now, we would like to extract the contract type (CDI, CDD, Alternance, etc) and the salary, IF THEY ARE MENTIONNED!
        try:
            salary_contract_element = driver.find_element(By.CLASS_NAME, 'css-1xkrvql').text
            
            # Once again, salary and contract types on Indeed share the same element, so we will have to separate them. An important notice here is that
            # salaries and contract types, IF they both exist in the element, are always separated by '-' (some jobs mention both, some mention only salaries,
            # some only contract while others don't mention any).
            
            # We begin by splitting the element by '-' if both elements are present.
            if ' - ' in salary_contract_element:
                salary_contract_parts = salary_contract_element.split(' - ')
                
            # Again, if both salaries and contracts are mentionned, we identify each part simply by identifying wich part of the element is digit (representing salary)
                if any(char.isdigit() for char in salary_contract_parts[0]):
                    job_details['salary'] = salary_contract_parts[0]
                    job_details['contract'] = salary_contract_parts[-1]
                elif any(char.isdigit() for char in salary_contract_parts[-1]):
                    job_details['salary'] = salary_contract_parts[-1]
                    job_details['contract'] = salary_contract_parts[0]

            # If there is no dash (meaning there is either salary or contract but not both, we identify whcih one is mentionned by scanning for digits in characters.
            # If digits is found, it means only salary is disclosed. If not, it means only contract type is mentionned.
            elif any(char.isdigit() for char in salary_contract_element):
                job_details['salary'] = salary_contract_element
                job_details['contract'] = "Not mentionned"
            elif not any(char.isdigit() for char in salary_contract_element):
                job_details['contract'] = salary_contract_element
                job_details['salary'] = "Not disclosed"
                
        # If we can't find the element to start with, it simply states that neither the salary nor the contract type is mentionned.
        except NoSuchElementException:
            job_details['salary'] = "Not disclosed"
            job_details['contract'] = "Not mentionned"

      
        # Extracting if there is possibility to work remotely.
        try:
            remote_work_text = driver.find_element(By.CLASS_NAME, 'css-17cdm7w').text
            job_details['remote_work'] = remote_work_text if remote_work_text else "Not mentioned" 
        except NoSuchElementException:
            job_details['remote_work'] = "Not mentioned"
        # The reason we use both 'try' and 'if' here is that sometimes our driver would not find the element associated with remote_work, so it will
        # return "Not mentionned". But other times, it will find the element but return empty characters for some reason. To avoid mistakes in the dictionary, 
        # it should also specify in this case "Not mentionned".
    
        
        # Extracting the url associated with the "apply now button"
        try:
            apply_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Continuer pour postuler (s\'ouvre dans un nouvel onglet)"]')
            job_details['url'] = apply_button.get_attribute('href')
        except NoSuchElementException:
            job_details['url'] = url  
            
    except Exception as e:
        print("Error scraping job details:", e)
    
    finally:
        driver.quit()  # Close the driver after scraping
    
    return job_details

In [8]:
jobs_scraped_correctly = 0
jobs_ignored = 0
all_job_details = []

# Iterate through each job details link
for job_link in job_details_links[:50]:
    job_details = scrape_job_details(job_link)
    if job_details:  
        all_job_details.append(job_details)
        jobs_scraped_correctly += 1
    else:
        jobs_ignored += 1
        

print("Number of jobs successfully extracted:", jobs_scraped_correctly)
print("Number of failed jobs extraction:", jobs_ignored)  

C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.105\chromedriver-win32/chromedriver.exe
C:\Users\user\.wdm\drivers\chr

##### Finally, we can transform our list containing all job data into a dataframe using ***pandas*** for easier readability and manipulation later on

In [10]:
w = pd.DataFrame(all_job_details)


# Remove duplicate jobs and drop NA values for consistency in the dataframe.
w = w.drop_duplicates()
w = w.dropna().reset_index(drop=True)
w

,title,company,postal_code,city,salary,contract,remote_work,url
0,Administrateur des bases de données microsoft ...,EURO-INFORMATION PRODUCTION,67,Strasbourg,De 35 000 € à 60 000 € par an,"CDI, Temps plein",Not mentioned,https://fr.indeed.com/applystart?jk=23d23734ba...
1,Data scientist (H/F),Prosol,69,Chaponnay,Not disclosed,CDI,Not mentioned,https://fr.indeed.com/applystart?jk=ecdd60b4f0...
2,Administrateur des bases de données postgresql...,EURO-INFORMATION PRODUCTION,59,Verlinghem,De 35 000 € à 60 000 € par an,"CDI, Temps plein",Not mentioned,https://fr.indeed.com/applystart?jk=dd600ef7df...
3,Data Analyst F/H,Nature et découvertes,78,Versailles,Not disclosed,CDI,Not mentioned,https://fr.indeed.com/applystart?jk=5b54e40844...
4,Ingénieur avant-vente Data Cybersécurité F/H,Orange,06,Biot,Not disclosed,Not mentionned,Not mentioned,https://fr.indeed.com/applystart?jk=be3c208880...
5,Consultant Débutant Data Analyst en Financemen...,KPMG,92,Courbevoie,Not disclosed,CDI,Not mentioned,https://fr.indeed.com/applystart?jk=23af065388...
6,Data Scientist (H/F),ACM GIE,67,Strasbourg,De 45 000 € à 65 000 € par an,"CDI, Temps plein",Not mentioned,https://fr.indeed.com/applystart?jk=909df8e117...
7,Chargé de domaine data - Analyste Qualité des ...,MACIF,79,Niort,De 40 000 € à 55 000 € par an,"CDI, Temps plein",Not mentioned,https://fr.indeed.com/applystart?jk=4b9a4f769d...
8,Data Analyst (H/F),HARRY HOPE,54,Nancy,35 000 € par an,CDI,Not mentioned,https://fr.indeed.com/applystart?jk=2ca172ca72...
9,Data analyst- H/F/X,Société Générale,92,La Défense,Not disclosed,Alternance,Not mentioned,https://fr.indeed.com/applystart?jk=1008b58302...


Printing the entire DataFrame would be too much. Therefore, we can use ".head(n)" to print the first n records

<font color="red">
    
 **Note:** It is important to note the presence of an irregularity in the code. Indeed, when we separate the city from the postal code by identifying which part of 'location' is digit and which isn't, a very small part of the postal codes get corrupted. The reason for this is that some cities are identified by their name followed by their 'arrondissement number' (for example Paris 1er, Lyon 2e). The result is the association of the number of the arondissement as the first number in the postal code (for example, if a job's location is identified as Paris 1er with a postal code of 75 or 75000, the postal code will appear in the list (and susbsequently in the dataframe) as 17 instead of 75)

<font>

## Part 2 : building the interface
In this next part, we will focus on building the interface. From this, users will be able to define specific criteria (job location, company name, keyword in job title, etc...) and search for job postings accordingly.
### A) Interface design
The interface design is divided into 2 parts. The first relates to searching jobs and directly seeing them, while the second relates to receiving an email containing the filtered jobs. We will present the interface creation code first, and then the functions that it uses. However, when running the code and testing it, **<font color="red"> IT IS IMPORTANT TO RUN ALL THE FUNCTIONS BELOW THIS CODE BEFORE RUNNING THE INTERFACE CODE <font color="red">**
#### 1. Job searching
The first step in creating an interface consists of designing it, which means we create different interactable objects inside of a window (called root). Specifically, we have decided to create 6 objects with which the user can interact:

- **Keyword as text entry** : this allows the user to specify a company's name (for example Orange) or a job title (for example Consultant).

- **Contract type as Combobox** : this allows the user to choose a contract type among dropdown options (CDI, CDD, Stage, Alternance, Contrat d'apprentissage or freelance)

- **City as text entry** : this allows the user to enter the city's name

- **Postal code as entry**: Same as city but for postal code (2 numbers only)

- **Possibility of remote work as Combobox**: same as contract type but for remote work. Options vary between 'Télétravail complet', 'Télétravail partiel' and 'Télétravail occasionnel'.

- **Search button**: when clicked, this allows the user to search for the job postings from our dataframe based on the selected criteria. To activate the search button so that it becomes interactive, we will refer to the 'search_jobs' function.

For each object, we will specify its title (for example Keyword, Contract type, ...), its type (Text entry, Combobox, ...) and its position in the window (using the grid function)

#### 2. Receiving Email
In the second part of the interface, we will upgrade the interface so that the user can choose receiving the filtered jobs by mail. The word 'choice' is very important. Indeed, we introduce a new interactable checkbox which displays additional parts of the interface **ONLY** when clicked. This is done when the user wishes to receive the jobs by email.

When the user clicks on the checkbox, 3 new objects appear:

- **Email as text entry** : allows user the enter his email
- 
- **Send jobs by mail as button** : when clicked, an message will be sent to the email entered, containing the job filtered based on the criteria

In [19]:
# Customizing apperance and colors of the interface using CustomTkinter
customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("green")

#Initializing the main window (in other terms the root)
root = customtkinter.CTk()
root.title("Job Search")

# X_var is essentially a StringVar object that acts as a container for the text entered by the user in the entry field represented by 
# X_entry. This allows to easily retrieve and manipulate the text entered by the user elsewhere in the code by accessing the value of X_var.

# Keyword
keyword_label = customtkinter.CTkLabel(root, text="Keyword:")
keyword_label.grid(row=0, column=0, padx=5, pady=2, sticky="w")
keyword_var = tk.StringVar(root)
keyword_entry = ttk.Entry(root, textvariable=keyword_var)
keyword_entry.grid(row=0, column=1, padx=5, pady=2)

# Contrat type
contracts_options = ["Any", "CDI", "CDD", "Stage", "Alternance","Contrat d'apprentissage","freelance"]  
contract_label = customtkinter.CTkLabel(root, text="Contract type :")
contract_label.grid(row=1, column=0, padx=5, pady=2, sticky="w")
contract_var = customtkinter.StringVar(root)
contract_dropdown = ttk.Combobox(root, textvariable=contract_var, values=contracts_options)
contract_dropdown.grid(row=1, column=1, padx=5, pady=2)
contract_dropdown.current(0)  # Set default selection to "Any"

# City
city_label = customtkinter.CTkLabel(root, text="City:")
city_label.grid(row=2, column=0, padx=5, pady=2, sticky="w")
city_var = tk.StringVar(root)
city_entry = ttk.Entry(root, textvariable=city_var)
city_entry.grid(row=2, column=1, padx=5, pady=2)

# Postal Code
postal_code_label = customtkinter.CTkLabel(root, text="Postal Code:")
postal_code_label.grid(row=3, column=0, padx=5, pady=2, sticky="w")
postal_code_var = tk.StringVar(root)
postal_code_entry = ttk.Entry(root, textvariable=postal_code_var)
postal_code_entry.grid(row=3, column=1, padx=5, pady=2)

# Remote work
remote_work_options = ['Any','Télétravail complet', 'Télétravail partiel', 'Télétravail occasionnel','Not mentioned']
remote_work_label = customtkinter.CTkLabel(root, text="Remote Work:")
remote_work_label.grid(row=4, column=0, padx=5, pady=2, sticky="w")
remote_work_var = tk.StringVar(root)
remote_work_dropdown = ttk.Combobox(root, textvariable=remote_work_var, values=remote_work_options)
remote_work_dropdown.grid(row=4, column=1, padx=5, pady=2)
remote_work_dropdown.current(0)  # Set default selection to "Any"

# Search button
search_button = customtkinter.CTkButton(root, text="Search", command=search_jobs)
search_button.grid(row=5, column=0, columnspan=2, padx=5, pady=5)

# Checkbox for receiving similar offers
receive_jobs_var = customtkinter.IntVar(root)
receive_jobs_checkbox = customtkinter.CTkCheckBox(root, text="Receive jobs by email", variable=receive_jobs_var, command=lambda: toggle_entries(email_label, email_entry,  send_bymail_button))
receive_jobs_checkbox.grid(row=6, column=0, columnspan=2, padx=5, pady=2, sticky="w")

# Enter Email
email_label = customtkinter.CTkLabel(root, text="Enter Email:")
email_var = tk.StringVar(root)
email_entry = ttk.Entry(root, textvariable=email_var)


# Button to send jobs by mail
send_bymail_button = customtkinter.CTkButton(root, text="Send Jobs by email", command=send_bymail)


toggle_entries(email_label, email_entry, send_bymail_button)

root.mainloop()

### B) Interface functions
#### Search jobs function
This function activates the 'search button' in the interface. It works as following:

1-  It saves the entries the users choose as selected_X (for example, if the user chooses 'CDI' as contract type, it save as selected_contract the choice of 'CDI' using the 'contract_var' in the code above, and so on).

***Reminder : X_var is essentially a StringVar object that acts as a container for the text entered by the user in the entry field represented by X_entry. This allows to easily retrieve and manipulate the text entered by the user elsewhere in the code by accessing the value of X_var***

2- It copies the dataframe containing all scraped jobs data.

3- Based on the user's entries (step 1), it filters jobs from the copied dataframe (step 2). 

4- Finally, it applies the filtered dataframe 'filtered_jobs' to the function 'display_job_listing' whcih we will see next

In [12]:
def search_jobs():
    selected_keyword = keyword_var.get()
    selected_contract = contract_var.get()
    selected_city = city_var.get()
    selected_postal_code = postal_code_var.get()
    selected_remote_work = remote_work_var.get()

    # Filter the DataFrame based on user choices            
    filtered_jobs = w.copy()
    
    if selected_keyword != "":
    # Filter based on the keyword in the 'title' column
        filtered_jobs = filtered_jobs[filtered_jobs['title'].str.contains(selected_keyword, case=False) | filtered_jobs['company'].str.contains(selected_keyword, case=False)]
    
    if selected_contract != "Any":
        filtered_jobs = filtered_jobs[filtered_jobs['contract'].str.contains(selected_contract)]
        
    if selected_city != "":
        # Convert both the entered city name and city names in the DataFrame to lowercase
        selected_city_lower = selected_city.lower()
        filtered_jobs = filtered_jobs[filtered_jobs['city'].str.lower().str.contains(selected_city_lower)]
    
    if selected_postal_code != "":
        # Filter based on the entered postal code
        filtered_jobs = filtered_jobs[filtered_jobs['postal_code'] == selected_postal_code]

            
    if selected_remote_work != "Any":
        if selected_remote_work == "Any":
            filtered_jobs = filtered_jobs[filtered_jobs['remote_work'].isin(['Télétravail complet', 'Télétravail partiel', 'Télétravail occasionnel','Not mentionned'])]
        else:
            filtered_jobs = filtered_jobs[filtered_jobs['remote_work'] == selected_remote_work]
   

    # Display the filtered jobs in a new window
    display_job_listings(filtered_jobs)

#### Display job listings function
This function displays the jobs that were filtered based on the user's criteria, and stored in 'filtered_jobs'. It works as the follwing:

1 - First creating a new window 'job_window", titled 'Job Listings', which is a secondary window to root.

2- Second, it displays the filtered jobs in a table like structure using the Teeview function (we can choose other formats sch as listbox but a table is more appropriate for our case). Te Treeview function retunr a table where each row represent a job, and each column an specific scraped element (such as Title, company name, etc...). 

3- An important element is the scraped url, which we will make use of using the 'open_url' function below to direct the user to the url when he clicks on the job in the Treeview. Some urls take the user to the company's website, while others redirect you to the Indeed job post. This is completely out of our control, since its Indeed that specify the destination of the URL.

In [13]:
def display_job_listings(filtered_jobs):
    # Create a new window
    job_window = tk.Toplevel(root)
    job_window.title("Job Listings")

    # Create a Treeview widget to display job listings
    job_tree = ttk.Treeview(job_window)
    job_tree['columns'] = ('company', 'contract', 'city', 'postal_code', 'salary', 'remote_work', 'url')
    # By default, Treeview stores a first primary column as a unique identifier. We will use this first column to store titles in it, not because it 
    # is a unique identifier, but simply because it is convenient. Then we will change its name to 'Title'
    
    # Define column headings
    job_tree.heading('#0', text='Title')
    job_tree.heading('company', text='Company')
    job_tree.heading('contract', text='Contract')
    job_tree.heading('city', text='City')
    job_tree.heading('postal_code', text='Postal Code')
    job_tree.heading('salary', text='Salary')
    job_tree.heading('remote_work', text='Remote Work')
    job_tree.heading('url', text='Go to website')

    # Insert job listings into the Treeview
    for idx, job in filtered_jobs.iterrows():
        title = job['title']
        company = job['company']
        contract = job['contract']
        city = job['city']
        postal_code = job['postal_code']
        salary = job['salary']
        remote_work = job['remote_work']
        url = job['url']

        # Insert job listing into the Treeview
        job_tree.insert('', 'end', text=title, values=(company, contract, city, postal_code, salary, remote_work, url))

    # Bind the event to open the respective URL when a row is clicked. It launches the 'open_url' function below.
    job_tree.bind('<ButtonRelease-1>', open_url)

    job_tree.pack(expand=True, fill=customtkinter.BOTH)

#### Open URL function
Simply, this function allows to extract the url of the respective job in the Treeview table and open it in the default browser (which is google in our case), when an event is launched. This event corresponds to the clicking of the row, as specfifed in the code above (job_tree.bind())

In [14]:
def open_url(event):
    item = event.widget.selection()[0] 
    url = event.widget.item(item, 'values')[-1]  # Get value from last column, which is associated with the url
    webbrowser.open_new(url)

#### Toggle entires function
The function below makes sure that the second part of the interface ('Enter Name', 'Enter Email', Save Criteria' and 'Send jobs by mai') appears only when the 'receive jobs by mail' checkbox is checked, and disappears when it is unchecked.

In [15]:
def toggle_entries(y1, y2, z1):
    if receive_jobs_var.get() == 1:
        y1.grid(row=8, column=0, padx=5, pady=5, sticky="w")
        y2.grid(row=8, column=1, padx=5, pady=5)
        z1.grid(row=9, column=1, columnspan=2, padx=5, pady=5)
    else:
        y1.grid_remove()
        y2.grid_remove()
        z1.grid_remove()

#### Filter job listing function
This function works exactly the same as the 'search_jobs' function, but istead of retunring the 'display_job_listings_ function, it simply returns the filtered jobs dataframe, to be used in the function above. Althought it is not optimal, it allows to filter jobs without neccessarily activating the 'search' button.

In [16]:
def filter_job_listings(dataframe):
    selected_keyword = keyword_var.get()
    selected_contract = contract_var.get()
    selected_city = city_var.get()
    selected_postal_code = postal_code_var.get()
    selected_remote_work = remote_work_var.get()

    # Filter the DataFrame based on user choices
    filtered_jobs = dataframe.copy()

    if selected_keyword != "":
        # Filter based on the keyword in the 'title' column
        filtered_jobs = filtered_jobs[filtered_jobs['title'].str.contains(selected_keyword, case=False) | filtered_jobs['company'].str.contains(selected_keyword, case=False)]

    if selected_contract != "Any":
        # Filter based on the contract type
        filtered_jobs = filtered_jobs[filtered_jobs['contract'].str.contains(selected_contract)]

    if selected_city != "":
        # Convert both the entered city name and city names in the DataFrame to lowercase
        selected_city_lower = selected_city.lower()
        filtered_jobs = filtered_jobs[filtered_jobs['city'].str.lower().str.contains(selected_city_lower)]

    if selected_postal_code != "":
        # Filter based on the entered postal code
        filtered_jobs = filtered_jobs[filtered_jobs['postal_code'] == selected_postal_code]

    if selected_remote_work != "Any":
        filtered_jobs = filtered_jobs[filtered_jobs['remote_work'].str.contains(selected_remote_work)]

    return filtered_jobs

#### Send personalized jobs by email
This function sends an email containing the filtered jobs. It allows to specify the sneder's email, receiver'email (which is the email entered by the user), the subject and a body (which in our case, contains the filtered jobs).

We use a gmail server since our email account is associatd with google. 

However, it is impotant to note a problem we faced:

**Problem :** server.login using the email's password did not work. Error message returned : could not connect 'wrong username or password'. 

**Reason :** as it turns out, google has removed the less secure app access feature which used to allow less secure apps (such as python) to connect
and send emails automatically. 

**Solution :** turn on 2-step-verification on the gmail account. Generate an App pasword, which is 16-digit password that google will generate allowing
complete access of apps to the gmail account. Insert the generated password in the 'pasword' slot in the server.login. 

In [17]:
def send_email(email, jobs):
    message = EmailMessage()
    message['from'] = "Job Alert <alouan.test@gmail.com>"
    message['to'] = email
    message['subject'] = "Job Alert: New Job Listings"
    message.set_content(jobs)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login('alouan.test@gmail.com', 'mnep kgfv ttux ybsj')
        server.send_message(message)
        server.quit()



When clicked, the 'send by email' button will activate this belowf unction, which allows to organize the filtered jobs in the dataframe as strings. In turn, when this function is called, it itself calls another function 'send_ma It allows to concatenate the filtered jobs in the dataframe into strings, and organizes the structure of the body of the mail. 

In [18]:
# Function to receive jobs my email
def send_bymail():
    # Get the email address from the entry widget
    email = email_entry.get()

    # Filter job listings based on the criteria
    filtered_jobs = filter_job_listings(w)

    # Check if there are any filtered jobs
    if not filtered_jobs.empty:
        # Convert filtered jobs DataFrame to a formatted string for email body
        jobs_str = ""
        for index, row in filtered_jobs.iterrows():
            job_info = f"Title: {row['title']}\nCompany: {row['company']}\nCity: {row['city']}\nSalary: {row['salary']}\nContract: {row['contract']}\nRemote Work: {row['remote_work']}\nURL: {row['url']}\n\n"
            jobs_str += job_info

        # Send email with job listings
        send_email(email, jobs_str)
        messagebox.showinfo("Success", "Job alerts sent successfully!") # message box that will pop up indicating the successful ssending of the job
                                                                        # by email
    else:
        messagebox.showinfo("No Jobs Found", "No jobs found based on the criteria.")